# Demo
Examples of how to use this repo

These are the commands for VPT with `p=50`:

VPT-deep
```bash
    --config-file configs/prompt/*.yaml
    MODEL.TRANSFER_TYPE "prompt" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.NUM_TOKENS "50" \
    MODEL.PROMPT.DROPOUT "0.0" 
```
   
VPT-shallow (we don't use dropout for VPT-shallow)
```bash
    --config-file configs/prompt/*.yaml
    MODEL.TRANSFER_TYPE "prompt" \
    MODEL.PROMPT.DEEP "False" \
    MODEL.PROMPT.NUM_TOKENS "50" \
    MODEL.PROMPT.DROPOUT "0.0" 
```

Other transfer protocols presented in the paper:

Full
```bash
    --config-file configs/finetune/*.yaml
```

Head-oriented methods:

- Linear:
```bash
    --config-file configs/linear/*.yaml
```

- MLP-3 (3 layer MLP):
```bash
    --config-file configs/linear/*.yaml \
    MODEL.MLP_NUM "2"
```

- Partial-1:
```bash
    --config-file configs/finetune/*.yaml \
    MODEL.TRANSFER_TYPE "partial-1"
```


Backbone-oriented methods:

- Sidetune:
```bash
    --config-file configs/linear/*.yaml
    MODEL.TRANSFER_TYPE  "side" 
```

- Bias: 
```bash
    --config-file configs/finetune/*.yaml \
    MODEL.TRANSFER_TYPE "tinytl-bias"
```

- Adapters with `r=128`:
```bash
    --config-file configs/finetune/*.yaml
    MODEL.ADAPTER.REDUCATION_FACTOR "128"
    MODEL.TRANSFER_TYPE "adapter" 
```

##  train.py
The main script is `train.py`. Note for VTAB data, this script handles the final runs with 1000 training data. See `tune_vtab.py` for the full tune + final runs settings. Here are some examples.

Note: it's recommended to directly use terminal for these command.

In [2]:
import torch

In [3]:
torch.cuda.device_count()

AttributeError: module 'torch' has no attribute 'cuda'

In [2]:
%%bash

source activate prompt

model_root=/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/pretrained/
data_path=/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/datasets/cub
output_dir=/home/yourenz/SynergisticEfficiency/experiments

python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "False" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitti16_imagenet21k" \
        SEED 42 \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}" 

[03/26 14:45:16 visual_prompt]: Rank of current process: 0. World size: 1
[03/26 14:45:16 visual_prompt]: Environment info:
-------------------  ----------------------------------------------------
Python               3.8.16 (default, Mar  2 2023, 03:21:46) [GCC 11.2.0]
ENV_MODULE           <not set>
PyTorch              1.7.1
PyTorch Debug Build  False
CUDA available       False
Pillow               9.3.0
cv2                  4.7.0
-------------------  ----------------------------------------------------
PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS=MKL, BUILD_TYPE=Release, CXX_FLAGS= -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -fopenmp -DNDEBUG 

Traceback (most recent call last):
  File "train.py", line 132, in <module>
    main(args)
  File "train.py", line 127, in main
    train(cfg, args)
  File "train.py", line 104, in train
    model, cur_device = build_model(cfg)
  File "/home/yourenz/SynergisticEfficiency/src/models/build_model.py", line 30, in build_model
    assert (
AssertionError: Cannot use more GPU devices than available


CalledProcessError: Command 'b'\nsource activate prompt\n\nmodel_root=/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/pretrained/\ndata_path=/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/datasets/cub\noutput_dir=/home/yourenz/SynergisticEfficiency/experiments\n\npython train.py \\\n        --config-file configs/prompt/cub.yaml \\\n        MODEL.TYPE "vit" \\\n        DATA.BATCH_SIZE "64" \\\n        MODEL.PROMPT.NUM_TOKENS "100" \\\n        MODEL.PROMPT.DEEP "False" \\\n        MODEL.PROMPT.DROPOUT "0.1" \\\n        DATA.FEATURE "sup_vitti16_imagenet21k" \\\n        SEED 42 \\\n        MODEL.MODEL_ROOT "${model_root}" \\\n        DATA.DATAPATH "${data_path}" \\\n        OUTPUT_DIR "${output_dir}" \n'' returned non-zero exit status 1.

In [ ]:
%%bash
# launch final training with five random seeds for VTAB-dmlab, sun397 and eurosat. The hyperparameters are the same from our paper.
model_root=<MODEL_ROOT>
data_path=<DATA_PATH>
output_dir=<OUTPUT_DIR>
        
# vtab-structured: dmlab
# base_lr = 1.0
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-dmlab" \
        DATA.NUMBER_CLASSES "6" \
        SOLVER.BASE_LR "0.25" \
        SOLVER.WEIGHT_DECAY "0.001" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

# vtab-natural: sun397
# base_lr = 25
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "128" \
        MODEL.PROMPT.NUM_TOKENS "5" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-sun397" \
        DATA.NUMBER_CLASSES "397" \
        SOLVER.BASE_LR "12.5" \
        SOLVER.WEIGHT_DECAY "0.0001" \
        SOLVER.TOTAL_EPOCH "100" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

# vtab-specialized: vtab-eurosat
# base_lr = 1
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-eurosat" \
        DATA.NUMBER_CLASSES "10" \
        SOLVER.BASE_LR "0.25" \
        SOLVER.WEIGHT_DECAY "0.001" \
        SOLVER.TOTAL_EPOCH "100" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

## Get results

In [1]:
import glob
import pandas as pd

from src.utils.vis_utils import get_df, average_df

LOG_NAME = "logs.txt"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
root = <MODEL_ROOT>
df_list=[]
for seed in ["42", "44", "82", "100", "800"]:
#     model_type = f"adapter_{r}"
    files = glob.glob(f"{root}/seed{seed}/*/sup_vitb16_imagenet21k/*/*/{LOG_NAME}")
    for f in files:
        df = get_df(files, f"seed{seed}", root, is_best=False, is_last=True)
        if df is None:
            continue
        df["seed"] = seed
    df_list.append(df)

df= pd.concat(df_list)
df["type"] = "VPT"
df

seed800: 100%|██████████| 3/3 [00:00<00:00, 252.79it/s]


,data,feature,lr,wd,total_params,tuned_params,tuned / total (%),batch_size,l-val_top1,l-test_top1,best_epoch,file,total_time,seed,type
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,46.88,76 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:03,42,VPT
0,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.00,38 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:57,42,VPT
1,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,52.57,14 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:57,42,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,99.5,46.25,85 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:09,44,VPT
0,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.54,41 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:38,44,VPT
1,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,49.03,32 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:53,44,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,46.14,65 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:04:18,82,VPT
1,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.67,41 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:42,82,VPT
0,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,52.45,8 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:51,82,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,47.41,76 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:15,100,VPT


Take average of 5 runs for each dataset:

In [4]:
# LR represents the base learning rate, not the scaled one.
f_df = average_df(df, metric_names=["l-test_top1"], take_average=True)
f_df

,data,feature,type,total_runs,l-test_top1,l-test_top1-std,lr,wd,total_params,tuned_params,tuned / total (%),batch_size,l-val_top1,total_time,seed
0,vtab-dmlab,sup_vitb16_imagenet21k,VPT,5,46.62,0.47,1.0,0.0010,86724870,926214,1.0680,64,100.0,0 days 01:06:03,42
2,vtab-eurosat,sup_vitb16_imagenet21k,VPT,5,96.15,0.45,1.0,0.0010,86727946,929290,1.0715,64,100.0,0 days 00:44:57,42
1,vtab-sun397,sup_vitb16_imagenet21k,VPT,5,51.57,1.40,25.0,0.0001,86150029,351373,0.4079,128,100.0,0 days 00:44:57,42


## tune*.py
Tune vtab or fgvc datasets. 

In [ ]:
%%bash
# Tune VTAB-caltech101 with VPT:
python tune_vtab.py \
    --train-type "prompt" \
    --config-file configs/prompt/cub.yaml \
    MODEL.TYPE "vit" \
    DATA.BATCH_SIZE "128" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.DROPOUT "0.1" \
    MODEL.PROMPT.NUM_TOKENS "10" \
    DATA.FEATURE "sup_vitb16_imagenet21k" \
    DATA.NAME "vtab-caltech101" \
    DATA.NUMBER_CLASSES "102" \
    DATA.DATAPATH <DATA_PATH> \
    MODEL.MODEL_ROOT <MODEL_ROOT> \
    OUTPUT_DIR <OUTPUT_PATH> 

# Tune CUB with VPT:
python tune_fgvc.py \
    --train-type "prompt" \
    --config-file configs/prompt/cub.yaml \
    MODEL.TYPE "vit" \
    DATA.BATCH_SIZE "128" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.DROPOUT "0.1" \
    MODEL.PROMPT.NUM_TOKENS "10" \
    DATA.FEATURE "sup_vitb16_imagenet21k" \
    DATA.DATAPATH <DATA_PATH> \
    MODEL.MODEL_ROOT <MODEL_ROOT> \
    OUTPUT_DIR <OUTPUT_PATH> 

## Backbone choices

- Swin-B

```bash
    MODEL.TYPE "swin" \
    DATA.FEATURE "swinb_imagenet22k_224"
```

- ResNet-50 (VPT with prompt location == pad)

```bash
    MODEL.TYPE "resnet" \
    DATA.FEATURE "imagenet_sup_rn50" \
    SOLVER.OPTIMIZER "sgd" \
    MODEL.PROMPT.LOCATION "pad" \
    MODEL.PROMPT.NUM_TOKENS "5" 
```

- ConvNeXt-Base (VPT with prompt location == pad)

```bash
    MODEL.TYPE "resnext" \
    DATA.FEATURE "imagenet22k_sup_rnx_base" \
    MODEL.PROMPT.LOCATION "pad" \
    MODEL.PROMPT.NUM_TOKENS "5" 
```

ViT with self-supervised pre-training objectives:
    
- MAE

```bash
MODEL.TYPE "ssl-vit" \
DATA.FEATURE "mae_vitb16"
```

- MoCo-v3

```bash
MODEL.TYPE "ssl-vit" \
DATA.FEATURE "mocov3_vitb" 
```